# MIPT DB HW 3

#### Libraries

In [19]:
# Импортируем модуль для подключения и работы с PostgreSQL:
import psycopg2 as pg

# Импортируем модуль для работы с данными:
import pandas as pd

#### PostgreSQL Connection

In [20]:
try:
    # Установливаем соединение с PostgreSQL:
    conn = pg.connect(dbname='postgres', host='localhost', user='postgres', password='Seven8ins', port=5432)
    # Создаем курсор для выполнения SQL-запросов:
    cursor = conn.cursor()
    # Выполняем простой SQL-запрос SELECT 1 для проверки соединения:
    cursor.execute("SELECT 1")
    # Если все прошло успешно, выводим сообщение об успешном соединении:
    print("Connection successful")


# Если произошла ошибка при подключении или выполнении запроса, выводим сообщение об ошибке:
except Exception as e:
    print("An error occurred:", e)

Connection successful


#### SQL queries

In [21]:
# Откатываем текущую транзакцию:
try:
    conn.rollback()
except:
    pass

* (1 балл) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [22]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT job_industry_category, COUNT(customer_id) AS customer_count
                FROM customer
                GROUP BY job_industry_category
                ORDER BY customer_count desc
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,job_industry_category,customer_count
0,Manufacturing,5593
1,Financial Services,5418
2,None,4592
3,Health,4207
4,Retail,2506
5,Property,1869
6,IT,1561
7,Entertainment,952
8,Argiculture,791
9,Telecommunications,504


* (1 балл) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [23]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT 
                    DATE_TRUNC('month', transaction_date)::date as month,
                        c.job_industry_category,
                        SUM(t.list_price) AS total_transaction_amount
                FROM transaction t
                INNER JOIN customer c ON t.customer_id = c.customer_id
                GROUP BY month, c.job_industry_category
                ORDER BY month, c.job_industry_category
                 """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,month,job_industry_category,total_transaction_amount
0,2017-01-01,Argiculture,1218386.96
1,2017-01-01,Entertainment,1794517.76
2,2017-01-01,Financial Services,10258743.88
3,2017-01-01,Health,8021999.44
4,2017-01-01,IT,3017934.36
...,...,...,...
115,2017-12-01,Manufacturing,8938950.44
116,2017-12-01,Property,3283369.60
117,2017-12-01,Retail,4294352.16
118,2017-12-01,Telecommunications,723871.96


* (1 балл) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [24]:
# Выполняем SQL-запрос:
cursor.execute("""
               SELECT t.brand, COUNT(*) AS online_orders_count
               FROM transaction t
               INNER JOIN customer c ON t.customer_id = c.customer_id
               WHERE c.job_industry_category = 'IT'
                    AND t.order_status = 'Approved'
                    AND t.online_order = 'True'
               GROUP BY t.brand
               ORDER BY online_orders_count DESC
               """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,brand,online_orders_count
0,Solex,2828
1,Norco Bicycles,2576
2,WeareA2B,2520
3,Giant Bicycles,2492
4,Trek Bicycles,2296
5,OHM Cycles,2184
6,None,224


* (2 балла) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

group by:

In [25]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT 
                    customer_id,
                    COUNT(transaction_id) AS transaction_count,
                    SUM(list_price) AS total_amount,
                    MAX(list_price) AS max_amount,
                    MIN(list_price) AS min_amount
                FROM transaction
                GROUP BY customer_id
                ORDER BY total_amount DESC, transaction_count desc
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,transaction_count,total_amount,max_amount,min_amount
0,2183,56,76285.28,2005.66,230.91
1,1129,52,73397.08,1992.93,290.62
2,1597,48,72210.72,2091.47,360.40
3,941,40,71593.84,2091.47,1057.51
4,2788,44,69035.76,2083.94,183.86
...,...,...,...,...,...
3489,2423,4,810.48,202.62,202.62
3490,3189,8,802.80,100.35,100.35
3491,2274,8,571.92,71.49,71.49
3492,2532,4,285.96,71.49,71.49


partition:

In [26]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT distinct customer_id, 
                        COUNT(transaction_id) OVER (PARTITION BY customer_id) AS transaction_count,
                        SUM(list_price) OVER (PARTITION BY customer_id) AS total_transaction_amount, 
                        MAX(list_price) OVER (PARTITION BY customer_id) AS max_transaction, 
                        MIN(list_price) OVER (PARTITION BY customer_id) AS min_transaction 
                        FROM transaction
                ORDER BY total_transaction_amount DESC, transaction_count desc
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,customer_id,transaction_count,total_transaction_amount,max_transaction,min_transaction
0,2183,56,76285.28,2005.66,230.91
1,1129,52,73397.08,1992.93,290.62
2,1597,48,72210.72,2091.47,360.40
3,941,40,71593.84,2091.47,1057.51
4,2788,44,69035.76,2083.94,183.86
...,...,...,...,...,...
3489,2423,4,810.48,202.62,202.62
3490,3189,8,802.80,100.35,100.35
3491,2274,8,571.92,71.49,71.49
3492,2532,4,285.96,71.49,71.49


* (2 балла) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [27]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT first_name, last_name
                FROM transaction
                JOIN customer as c on c.customer_id = transaction.customer_id
                GROUP BY first_name, last_name, c.customer_id
                ORDER BY SUM(list_price) ASC
                LIMIT 1
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,first_name,last_name
0,Hamlen,Slograve


In [28]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT first_name, last_name
                FROM transaction
                JOIN customer as c on c.customer_id = transaction.customer_id
                GROUP BY first_name, last_name, c.customer_id
                ORDER BY SUM(list_price) DESC
                LIMIT 1
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,first_name,last_name
0,Jillie,Fyndon


* (1 балл) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

In [29]:
# Выполняем SQL-запрос:
cursor.execute("""
                SELECT *
                FROM (
                    SELECT *, 
                        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY transaction_date ASC) AS rn
                        FROM transaction
                    ) AS ranked_transactions
                WHERE rn = 1
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,rn
0,9785,72,1,2017-01-05,False,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,1
1,2261,1,2,2017-05-04,True,Approved,Giant Bicycles,Standard,medium,medium,1403.50,954.82,1
2,10302,33,3,2017-02-23,False,Approved,Giant Bicycles,Standard,medium,small,1311.44,1167.18,1
3,12441,95,4,2017-04-03,False,Approved,Giant Bicycles,Standard,medium,large,569.56,528.43,1
4,2291,23,5,2017-03-03,True,Approved,Norco Bicycles,Mountain,low,small,688.63,612.88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3489,8276,18,3497,2017-09-01,True,Approved,Solex,Standard,medium,medium,575.27,431.45,1
3490,13469,12,3498,2017-02-03,True,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,1
3491,2794,62,3499,2017-01-12,False,Approved,Solex,Standard,medium,medium,478.16,298.72,1
3492,6309,69,3500,2017-01-09,True,Approved,Giant Bicycles,Road,medium,medium,792.90,594.68,1


* (2 балла) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).

In [30]:
# Выполняем SQL-запрос:
cursor.execute("""
                WITH new_tab AS (
                                SELECT customer_id, transaction_date, 
                                        LAG(transaction_date) OVER(PARTITION BY customer_id ORDER BY transaction_date), 
                                                transaction_date - LAG(transaction_date) 
                                        OVER(PARTITION BY customer_id ORDER BY transaction_date) AS difference_date
                                FROM transaction t 
                                ),
                   new_tab_2 AS (
                                SELECT first_name , last_name, job_title, difference_date, 
                                        RANK() OVER(ORDER BY difference_date DESC) AS rn
                                FROM new_tab
                                JOIN customer ON customer.customer_id = new_tab.customer_id 
                                WHERE difference_date IS NOT NULL
                                )
                SELECT DISTINCT first_name , last_name, job_title, difference_date
                FROM new_tab_2
                WHERE rn = 1
                """)

# Извлекаем все строки из результата запроса:
rows = cursor.fetchall()

# Получаем имена столбцов из описания курсора:
colnames = [desc[0] for desc in cursor.description]

# Преобразуем строки в DataFrame и устанавливаем имена столбцов:
df = pd.DataFrame(rows, columns=colnames)

# Отображаем DataFrame:
display(df)

# Подтверждаем транзакцию:
conn.commit()

,first_name,last_name,job_title,difference_date
0,Susanetta,None,Legal Assistant,357


In [31]:
# Закрываем курсор
cursor.close()

# Закрываем соединение с базой данных
conn.close()